# 5. Labor: Egyszerű Neurális Hálózat implementációja

A gyakorlat során az eddig tanult ismeretekre támaszkodva implementáljuk a lehető legegyszerűbb, egy rejtett réteget tartalmazó (nem deep) MLP (multi-layer preceptron) vagy FF (feed-forward) neurális hálót.

## A XOR probléma

A XOR vagy kizáró vagy művelete egy tipikus iskolapéldája a neurális hálók lehetőségeinek demonstrációira. Ugyan a XOR kérdés könnyen implementálható `if` elágazások segítségével, az adatokat elválasztó nem lineáris (de még csak nem is egy fügvénnyel leírható) határvonal gyakorlatilag megoldhatatlan a hagyományos statisztikai / regressziós modellek segítségével.

## A perceptron

Vizsgáljuk meg az eddig alkalmazott Logisztikus regressziós modell struktúráját. Ha grafikusan ábrázolni szeretnénk, az alábbi alakot kapjuk:

<!--
<center><img src="img/perceptron.svg" width="300"></center>
-->
<center><img src="https://drive.google.com/uc?export=view&id=1vC5wNfMRNwx8ZRY7h6HLDVHfaeN3ASA1" width="600"></center>


ahol

$$ s = \sum_{i=1}^n(w_i \cdot x_i); \qquad \hat{y} = a(s) $$

A modellünk bemenetei az $x_1$ - $x_n$ változók, az ezekhez rendelt súlyok (modellparaméterek) a $w_1$ - $w_n$ értékek, a regressziós értéket valószínűséggé konvertáló `a()`, u.n. aktivációs függvény pedig esetünkben a _sigmoid_ függvény volt. A modell kimenete a $\hat{y}$ valószínűség érték. Ezt a struktúrát _perceptronnak_ is szokás nevezni. Kis absztrakciós késséggel nem nehéz belátni, hogy alakjában és funkciójában nagyon hasonlít egy neuron működésére: a dendriteken beérkező elektromos ingerek súlyozott összege alapján a neuron testében (soma) adott szint elérésekor aktivációs potenciál keletkezik, amelyet a neuron az axonon keresztül továbbít a következő neuronok felé.

<!--
<center><img src="img/neuron.svg" width="600"></center>
-->
<center><img src="https://drive.google.com/uc?export=view&id=1AG4xBI_j4rBXvcM-ptJD6G2n8P63s6W_" width="600"></center>



A logisztikus regressziót implementáló perceptront több rétegben alkalmazva megkapjuk a multilayer perceptron / feed forward neural network arkhitektúrát:
<!--
<center><img src="img/mlp.svg" width="600"></center>
-->
<center><img src="https://drive.google.com/uc?export=view&id=1ZOXMS7y5cOAmB8Mwq7NN6k1Wlo-0CYFw" width="600"></center>


## Forward propagation - predikció neurális hálóval

A neurális hálók esetén a bemeneti adatainkat a bemeneti rétegen elindítva, az információ rétegről rétegre halad át a modellen, az információ a bemenetektől a kimenetig előrefele terjed, *'propagál'*. Vizsgáljuk meg, hogy hogyan fognak alakulni az egyes mátrixok méretei, miközben ez a folyamat végbemegy.

A bemeneti mátrixunk az eddigiekkel megegyezően néz ki, minden sora egy-egy külön bemeneti adatpont, oszlopai tartalmazzák az egyes bementi változókat:

$$
\underset{[m \ \times \ n(0)+1]}{\mathbf{X}} = \left[
	\begin{array}{ccccc}
 		1 & x_{1,1} & x_{1,2} & \ldots & x_{1,n(0)}\\
		1 & x_{2,1} & x_{2,2} & \ldots & x_{2,n(0)}\\
 		\vdots & \vdots & \vdots & \ddots & \vdots\\
        1 & x_{m,1} & x_{m,1} & \ldots & x_{m,n(0)}\\
	\end{array}	\right]
$$
ahol $n0$ a bemeneti réteg neuronjainak a száma.

A súlyokat tartalamzó mátrixból eddig csak egy darab szerepelt, annak pedi egy darab oszlopa volt. Az MLP struktúra esetén minden réteghez (leszámítva a bemeneti réteget) külön mátrix tartozik. Minden rétegben, neurononként egy oszlop tartalmazza az adott súlyokat, az egyes oszlopok az adott réteg adott neuronjaihoz tartoznak:

$$
\underset{[n(k-1)+1 \ \times \ n(k)]}{\mathbf{W}^{(k)}} = \left[
	\begin{array}{ccccc}
 		w_{0,1} & w_{0,2} & \ldots & w_{0,n(k))}\\
		w_{1,1} & w_{1,2} & \ldots & w_{1,n(k)}\\
 		\vdots & \vdots & \ddots & \vdots\\
        w_{n(k-1),1} & w_{n(k-1),2} & \ldots & w_{n(k-1),n(k)}\\
	\end{array}\right]
$$
ahol $n(k)$ az n. rétegben található neuronok száma, $n(k-1)$ pedig az azt megelőző réteg neuronjainak száma.

A $k$ réteg kimeneteit a már ismert logisztikus regressziónak megfelelő sémával számolhatjuk:
$$\underset{[m \ \times \ n(k)]}{\mathbf{\hat{Y}}^{(k)}} = a\left(\underset{[m \ \times \ n(k)]}{\mathbf{S}^{(k)}}\right) = a\left(\underset{[m \ \times \ n(k-1)+1]}{\mathbf{X}^{(k)}} \cdot \underset{[n(k-1)+1 \ \times \ n(k)]}{\mathbf{W}^{(k)}} \right)$$
$$\underset{[m \ \times \ n(k-1)+1]}{\mathbf{X}^{(k)}} = \underset{[m \ \times \ 1]}{BIAS} + \underset{[m \ \times \ n(k-1)]}{\mathbf{Y}^{(k-1)}}$$
ahol $a$ az hálóban (vagy adott rétegben) haszált aktivációs függvény. A tárgy keretében csak a sigmoid függvénnyel foglalkoztunk, mint aktivációs függvény, de számos másik létezik, amelyek közül szinte akármelyik szabadon alkalmazható. Az egyetlen megkötés általában az utolós rétegnél van, ahol a költségfüggvényhez hasonlóan egyeztetni kell az aktivációs függvényt a feladattal, hiszen ez határozza meg, hogy  a kimenet milyen tartományú lehet.:
- regresszió --> MSE & identitás aktivációs függvény
- bináris klasszifikáció --> BCE & sigmoid aktivációs függvény

Egy egyszerű, 2 bemeneti neuront és egy  rejtett rétegben 3 neuront tartalmazó bináris klasszifikációt megvalósító háló esetén a mátrixok méretének alakulása a forward propagation folyamán:
- a rejtett rétegre
$$ \underset{m \ \times \ 3}{\mathbf{X^{(1)}}} =  \underset{m \ \times \ 1}{BIAS} + \underset{m \ \times \ 2}{\mathbf{X}} $$
$$ \underset{m \ \times \ 3}{\mathbf{S^{(1)}}} = \underset{m \times 3}{\mathbf{X^{(1)}}} \times \underset{ 3 \ \times \ 3}{\mathbf{W^{(1)}}} $$
$$ \underset{m \ \times \ 3}{\mathbf{\hat{Y}}^{(1)}} = sigmoid(\underset{m \ \times \ 3}{{\mathbf{S}^{(1)}}}) $$
- a kimeneti rétegre
$$ \underset{m \times 4}{\mathbf{X^{(2)}}} = \underset{[m \ \times \ 1]}{BIAS} + \underset{m \ \times \ 3}{\mathbf{\hat{Y}}^{(1)}} $$
$$ \underset{m \ \times \ 1}{\mathbf{S^{(2)}}} = \underset{m \times 4}{\mathbf{X^{(2)}}} \times \underset{ 4 \ \times \ 1}{\mathbf{W^{(2)}}} $$
$$ \underset{m \ \times \ 1}{\mathbf{\hat{Y}}^{(2)}} = sigmoid(\underset{m \ \times \ 1}{{\mathbf{S}^{(2)}}}) $$
Az utolsó rétegünk kimenete maga a modell által adott becslés.
$$ \underset{m \ \times \ 1}{\mathbf{\hat{Y}}} = \underset{m \ \times \ 1}{\mathbf{\hat{Y}}^{(2)}} $$



## BackPropagation - neurális háló tanítása

A neurális hálók esetén a gradiens módszer elve nem változik. Az egyes súlyokat a költségfüggvény adott súly szerinti parciális deriváltjának alapján módosítjuk, azonban a gradiens számítása némileg komplikáltabb. Számítására a lánc szabályt használhatjuk. Nézzük meg a gradiens számítását a kimeneti réteghez tartozó súlyokra. A kimeneti réteg súlyaihoz tartozó parciális derivált a láncszabály alapján általánosan, $p$ az utolsó (kimeneti) réteg:

$$ \frac{\partial C}{\partial w^{(p)}_{ij}} =
\frac{\partial C}{\partial \hat  y_j} \frac{\partial \hat y_j}{\partial w^{(p)}_{ij}} = 
\frac{\partial C}{\partial \hat  y_j} \frac{\partial \hat  y_j}{\partial s^{(p)}_j} \frac{\partial s^{(p)}_j}{\partial w^{(p)}_{ij}} $$

Az előzőekben defináltuk, hogy

$$  \hat  y_j = \hat y^{(p)}_{j}; \qquad \hat y^{(k)}_{j}=a(s^{(k)}_j); \qquad s^{(k)}_j=\sum_{i=1}^n x^{(k)}_i w^{(k)}_{ij}; $$

amiből
$$ \frac{\partial \hat  y_j}{\partial s^{(p)}_j} = a'(s^{(p)}_j); \qquad \frac{\partial s^{(p)}_j}{\partial w^{(p)}_{ij}} = x^{(p)}_i$$

Ezek alapján az utolsó réteghez tartozó parciális deriváltak:
$$ \frac{\partial C}{\partial w^{(p)}_{ij}} =  \frac{\partial C}{\partial \hat  y_j} a'(s^{(p)}_j) x^{(p)}_i = \delta^{(p)}_j x^{(p)}_i; \qquad  \delta^{(p)}_j = \frac{\partial C}{\partial \hat  y_j} a'(s^{(p)}_j) $$

A több bemenetet figyelembe véve és mátrixos alakra felírva:

$$ \boxed{ \frac{\partial C}{\partial \mathbf{W}^{(p)}} =  \left(\mathbf{X^{(p)}_i}\right)^T \left( \frac{\partial C}{\partial \mathbf{\hat  Y}} a'(\mathbf{S}^{(p)}) \right)  = \left(\mathbf x^{(p)}\right)^T \cdot \mathbf \delta^{(p)} } $$


A *BCE* költségfüggvény és a *sigmoid* aktiváció együttes használatakor
$$ \boxed{\mathbf \delta^{(p)} = \frac{1}{m} \left(\mathbf{\hat Y} - \mathbf{Y} \right) = \frac{1}{m} \left(sigmoid(\mathbf{X}^{(p)}\cdot \mathbf{W}^{(p)} ) - \mathbf{Y} \right)}$$

Látható, hogy visszakaptuk az egyzserű logisztikus regresszió esetén alkalmazott gradiensvektort. A $k$ rejtett réteg esetében a gradiensek számítása a következőként írható fel:

$$ \frac{\partial C}{\partial w^{(k)}_{ij}} =
\frac{\partial C}{\partial y^{(k)}_{j}} \frac{\partial y^{(k)}_{j}}{\partial w^{(k)}_{ij}} =
\frac{\partial C}{\partial y^{(k)}_{j}} \frac{\partial y^{(k)}_{j}}{\partial s^{(k)}_{j}}\frac{\partial s^{(k)}_{j}}{\partial w^{(k)}_{ij}} =
\frac{\partial C}{\partial y^{(k)}_j} a'(s^{(k)}_j) x^{(k)}_i=
\delta^{(k)}_j x^{(k)}_i; $$

Mátrixos formában:
$$ \boxed{ \frac{\partial C}{\partial \mathbf{W}^{(k)}} = (\mathbf{X}^{(k)})^T \cdot \mathbf{\delta^{(k)}} }$$

A $k$ réteghez tartozó delta tag számítása pedig

$$ \mathbf{\delta^{(k)}} = \frac{\partial C}{\partial \mathbf{\hat  Y^{(k)}}} a'(\mathbf{s^{(k)}}) = \frac{\partial C}{\partial \mathbf{\hat  Y^{(k+1)}}} \frac{\partial \mathbf{\hat  Y^{(k+1)}}}{\partial \mathbf{\hat  Y^{(k)}}}a'(\mathbf{S^{(k)}}) = \frac{\partial C}{\partial \mathbf{\hat  Y^{(k+1)}}} \frac{\partial \mathbf{\hat  Y^{(k+1)}}}{\partial \mathbf{\hat  S^{(k+1)}}} \frac{\partial \mathbf{ S^{(k+1)}}}{\partial \mathbf{\hat  Yy^{(k)}}}a'(\mathbf{S^{(k)}})$$

$$\frac{\partial \mathbf{\hat  Y^{(k+1)}}}{\partial \mathbf{ S^{(k+1)}}} = a'(\mathbf{S^{(k+1)}})$$
$$\frac{\partial \mathbf{ S^{(k+1)}}}{\partial \mathbf{\hat  Y^{(k)}}} = \frac{\partial \mathbf{ S^{(k+1)}}}{\partial \mathbf{\hat  X^{(k+1)}}} = (\mathbf{W}^{(k+1)})^T$$

$$\boxed{ \mathbf{\delta^{(k)}} = \frac{\partial C}{\partial \mathbf{\hat  Y^{(k+1)}}} a'(\mathbf{S^{(k+1)}}) (\mathbf{W}^{(k+1)})^T a'(\mathbf{S^{(k)}}) = \left( \mathbf \delta^{(i+1)} \cdot \left( \mathbf W^{(i+1)} \right)^T \right) a'\left(\mathbf s^{(i)} \right) }$$

Látszik, hogy az implementációhoz mindenképp szükésg van az aktivációs függvény deriváltjára is. A sigmoid függvény esetében:
$$ \boxed{ sigmoid'(z) = sigmoid(z)(1-sigmoid(z))} $$

## 00: Könyvtár importálások

Első lépésként importáljuk a feladat megoldása során használt könyvtárakat. Esetünkben ezek a következők lesznek:
- Numpy a matematikia műveletek elvégzéséhez
- Pandas az adatok beolvasásához és kezeléséhez
- MatPlotLib.pyplot az eredményeink ábrázolásához
- Plotly Express interaktív vizualizációhoz

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Használjuk ezeket sötét téma esetén
plt.style.use('dark_background')
styleTemplate = 'plotly_dark'

# Használjuk ezeket világos téma esetén
#plt.style.use('default')
#styleTemplate = 'plotly_white'

## 01: Adatgenerálás
Az eddigiekkel eltérő módon most nem egy előre adott adatokkal dolgozunk. A XOR probléma adatstruktúrája viszonylag egyszerű, az adatokat magunknak generálhatjuk. Így tetszőleges számú adatot használhatunk.

In [ ]:
# GENERATE XOR DATA
nSamples = 100 # total samples = 4*nSamples
clusters = [[-0.5, -0.5, 0], [-0.5, 0.5, 1], [0.5, -0.5, 1], [0.5, 0.5, 0]]
std = 0.25
rng = np.random.default_rng(42)

X = np.ones([4*nSamples, 2])
Y = np.ones([4*nSamples, 1])
for count, params in enumerate(clusters):
    X[count*nSamples:(count+1)*nSamples, 0] = rng.normal(params[0], std, nSamples)
    X[count*nSamples:(count+1)*nSamples:,1] = rng.normal(params[1], std, nSamples)
    Y[count*nSamples:(count+1)*nSamples] = params[2]

## 02: Adatfelfedezés

A szintetikus, generált adatok esetén is érdemes előzetes adatfelfedezést / adatvizualizációt alkalmazni. Így ellenőrizni tudjuk, hogy biztodan oylan adatokat generáltunk-e, mint szerettünk volna.

In [ ]:
plt.figure(figsize=(6, 6))
falseData = X[Y[:,0] == 0, :]
trueData = X[Y[:,0] == 1, :]

plt.scatter(falseData[:, 0], falseData[:, 1], marker='o', c="r", label="False")
plt.scatter(trueData[:, 0], trueData[:, 1], marker='o', c="g", label="True")

plt.title("Generált XOR data")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend(loc='lower left')

plt.show()

A generált adataink ebben az esetben már megfelelnek a tanításra, a tartományukból adódóan normalizálásra nincs szükség. A bias tag implementációját most a modellen belül oldjuk meg.

## 03: Modell implementálása

Először implementáljuk az aktivációs függvényként használt `sigmoid()` függvényt. Mivel a hálónk tanításához szükségünk lessz a sigmoid deriváltjának számítására is, az implementációnkba ezt is belefoglaljuk.

**Feladat:** implementálja a `sigmoid()` aktivációs függvényt, amely második bementeként egy `bool` értéket vár, ami alapján vagy a *sigmoid* függvényt vagy annak a deriváltját számítja.

In [ ]:
def sigmoid(z, derivate = False):       # Alapértelmezett a rendes sigmoid érték számítása
######################################

######################################
    return g

Az implementációnkat most vizuálisan ellenőrizzük.

In [ ]:
x = np.linspace(-6, 6, 100)
y = sigmoid(x)
dy = sigmoid(x, derivate = True)

fig = go.Figure()
fig.add_trace(go.Scatter(x = x, y = y, mode='lines', name ='sigmoid'))
fig.add_trace(go.Scatter(x = x, y = dy, mode='lines', name = 'derivate of sigmoid'))

fig.update_layout(
    template='plotly_dark',
    xaxis_title = "z",
    title = "Sigmoid függvény",
    width=600,
    height=320,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y = 1.02,
        xanchor="center",
        x = 0.5)
)

fig.update_traces(line=dict( width=3))
fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey', zeroline=True, zerolinewidth=3, zerolinecolor='grey')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey', zeroline=True, zerolinewidth=3, zerolinecolor='grey')

fig.show()

Szintén külön függvényként definiáljuk a bemeneti mátrix biassal való kiegészítését.

In [ ]:
def addBias(z):
    return np.hstack([np.ones([z.shape[0] ,1]), z])

Bias kiegészítés tesztelése:

In [ ]:
testX = np.array([[0.3146, -0.65432, 0.24], [-1.0123, -0.4215, -0.12412], [0.2351, 0.7533456, 2.346]])
print(addBias(testX))

A neurális háló implementálásához és tanításához szükséges metódusokat és adattagokat most egy osztályba rendezve implementáljuk. A még nem üres metódusokat egy `pass` utasítással feltöltve elkerülhetjük, hogy a futtatás során hibát jelezzen, így egyenként ellenőrizhetjük a metódusainkat.

**Feladat:** implementálja a háló `forwardProp()` metódusát, amely elvégzi az adott bemeneti adatokra a előreterjesztés lépését! A metódus töltse fel az osztály belső X[k] és Yhat[k] listákat a megfelelő értékekkel! Az implementáció ellenőrzésére használja a forwardProp tesztelő cellát az osztálydefiníció alatt.

**Feladat:** implementálja a háló `backProp()` metódusát, amely a backPropagation módzser segítségével kiszámolja az egyes súlyokhoz tartozó gradienseket. Az implementáció ellenőrzésére használja a backProp tesztelő cellát az osztálydefiníció alatt.

In [ ]:
class FeedForwardNN:
    def __init__(self, layerSizes, activationFunction, seed):
        self.layerSizes = layerSizes            # Háló konfigurációja
        self.a = activationFunction             # Aktivációs függvény (később más aktivációs függvény is alkalmazható)
        self.noLayers = len(layerSizes)-1       # Rétegek száma (iterációhoz, bemeneti réteg nélkül)
        self.inputSize = layerSizes[0]          # Bemeneti változók (input feature) száma
        self.Yhat = []                             # Lista az egyes rétegek kimenetének tárolására
        self.X = []                             # Lista az egyes rétegek bemenetének tárolására
        self.W = []                             # Lista a rétegekhez tartozó súlyoknak
        self.initWeights(seed)                  # Súlyok inicializálása
    
    def initWeights(self, seed = None):
        rng = np.random.default_rng(seed)
        # Weight matrix dimension are based on number of neurons in previous and current layer
        self.W = [rng.uniform(low = -1, high = 1, size = [self.layerSizes[i]+1, self.layerSizes[i+1]]) for i in range(self.noLayers)]

    def checkInputSize(self, X):
        if X.shape[1] != self.inputSize:
            raise ValueError('Unexpected number of input features! Expected {} features but got {}.'.format(self.inputSize, X.shape[1])) 

    def forwardProp(self, X):
        self.X = [[] for i in range(self.noLayers)]
        self.Yhat = [[] for i in range(self.noLayers)]
    ######################################
        pass
    ######################################

    def predict(self, X):
        self.checkInputSize(X)
        self.forwardProp(X)
        return self.Yhat[-1]

    def costBCE(self, X, Y):
        eps = 10e-15
        self.forwardProp(X)
        return np.mean(-Y*np.log(self.Yhat[-1]+eps)-(1-Y)*np.log(1-self.Yhat[-1]))

    def backProp(self, trueY):
    ######################################
        pass
    ######################################
        #return dW

    def updateWeights(self, learning_rate, dW):
        for i in range(self.noLayers):
            self.W[i] = self.W[i] - learning_rate * dW[i]

    def fit(self, X, Y, learning_rate, epochs):
        C_history = np.zeros([epochs+1])
        C_history[0] = self.costBCE(X, Y)    # Forwardprop megtörténik
        print('''
        \%\%\% ------- TANÍTÁS ------- \%\%\%
        ''')
        for i in range(epochs):
            dW = self.backProp(Y)
            self.updateWeights(learning_rate, dW)
            C_history[i+1] = self.costBCE(X,Y) # Forwardprop megtörténik

            if ((i+1) % 250) == 0:
                
                print('Epoch {} / {} completed. Cost value:{}'.format(i+1, epochs, C_history[i+1])) 

        return C_history

Teszteljük az egyes metódusokat, hogy megbizonyosodjunk az elvárt működésről!

In [ ]:
layerSizes = [2, 3, 1]
seed = 42
testNN = FeedForwardNN(layerSizes, sigmoid, seed)

In [ ]:
# Test initweights
print('''Expected initial weights with layer sizes {0} and random seed {1}:
[array([[ 0.5479121 , -0.12224312,  0.71719584],
       [ 0.39473606, -0.8116453 ,  0.9512447 ],
       [ 0.5222794 ,  0.57212861, -0.74377273]]), array([[-0.09922812],
       [-0.25840395],
       [ 0.85352998],
       [ 0.28773024]])]'''.format(layerSizes, seed))
print('''Actual initial weights with layer sizes {0} and random seed {1}:
{2}'''.format(layerSizes, seed, testNN.W))

In [ ]:
# Test checkInputSize
testX = np.array([[0.3146, -0.65432, 0.24], [-1.0123, -0.4215, -0.12412], [0.2351, 0.7533456, 2.346]])
try:
    testNN.checkInputSize(testX)
except ValueError as ex:
    print(ex)

In [ ]:
# Test forwardProp & Predict
testX = np.array([[0.3146, -0.65432], [-1.0123, -0.4215], [0.2351, 0.7533456]])
testPred = testNN.predict(testX)
print('''Expected prediction for test parameters:
[[0.56445555]
 [0.610119  ]
 [0.58247854]]'''.format(layerSizes, seed))
print('''Actual prediction for test parameters:
{0}'''.format(testPred))

In [ ]:
# Test cost function
testX = np.array([[0.3146, -0.65432], [-1.0123, -0.4215], [0.2351, 0.7533456]])
testY = np.array([[0], [1], [1]])

testCost = testNN.costBCE(testX, testY)

print('''Expected cost value for test parameters:
0.6219075363927603''')

print('''Actual prediction for test parameters:
{0}'''.format(testCost))

In [ ]:
# Test backprop
testX = np.array([[0.3146, -0.65432], [-1.0123, -0.4215], [0.2351, 0.7533456]])
testY = np.array([[0], [1], [1]])

testNN.forwardProp(testX)
testdW = testNN.backProp(testY)

print('''Expected cost value for test parameters:
[array([[ 0.00351405, -0.02095111, -0.01093665],
       [-0.0105734 ,  0.03069215,  0.00971723],
       [ 0.00944793, -0.034079  , -0.0086122 ]]), array([[-0.0809823 ],
       [-0.05584407],
       [-0.09301554],
       [ 0.00406624]])]''')

print('''Actual prediction for test parameters:
{0}'''.format(testdW))

## 04: Modell tanítása

Amennyiben minden tesztünkön megfelelő eredményt kaptunk, végezzük el a háló tanítását a `.fit()` metódus segítségével, és ábrázoljuk a költségfüggvény alakulását.

In [ ]:
learning_rate = 0.5
trainNN = FeedForwardNN([2, 10, 1], sigmoid, 42)

C_history = trainNN.fit(X, Y, learning_rate, 3000)
plt.plot(range(C_history.size), C_history)

## 05: Modell értékelése

A beépített `predict()` függvénynek köszönhetően a vizualizálhatjuk az hálónk döntési stratégiáját, vizualizálhatjuk a működést 2D-ben contúrvonalakkal, vagy 3D-ben a teljes illesztett felületet ábrázolva.

In [ ]:
plt.figure(figsize=(6, 6))
falseData = X[Y[:,0] == 0, :]
trueData = X[Y[:,0] == 1, :]

plt.scatter(falseData[:, 0], falseData[:, 1], marker='o', c="r", label="False")
plt.scatter(trueData[:, 0], trueData[:, 1], marker='o', c="g", label="True")

x1 = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 200)    # grid létrehozása
x2 = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 200)    # második paraméter

z=np.zeros((len(x1),len(x2)))                          # eredményváltozó 1 inicializálása

for i in range(len(x1)):                                 # valószínűség számolása a teljes háló felett
    for j in range(len(x2)):     
        testPoint = np.array([[x1[i], x2[j]]])
        z[i,j] = trainNN.predict(testPoint)

plt.contour(x1, x2,z.transpose(), 3)                                  # kirajzoljuk contour plottal a döntési határt                                # kirajzoljuk contour plottal a döntési határt


plt.title("XOR becslés")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend(loc='lower left')

plt.show()

A beépített `predict()` fügvénynek köszönhetően a teljes illesztett felületet is vizualizálhatjuk.

In [ ]:
# Ábrázolás Plotly-val
fig = go.Figure()

# A magyarázott változót transzponálni kell a helyes megjelenítésért.
fig.add_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=Y[:,0], mode= "markers"))
fig.add_trace(go.Surface(x=x1, y=x2, z=z.T, colorscale ='Blues'))

#Plot formázása
fig.update_layout(
    title = "XOR becslés",
    scene = dict(
        xaxis_title = "x1",
        yaxis_title = "x2",
        zaxis_title = "Prob"),
    template=styleTemplate,
    width=750,
    height=500,
)

#Plot megjelenítése
fig.show()

# 06: További tesztelés

A gyakorlat keretében a tesztelést csak az illesztett felület vizualizásársa, illetve a költség alakulására korlátoztuk. Az eddig tanult egyéb ismeretek alapján érdemes egyéni munkában a hálót módosítani, kiegészítgetni, pl.: accuracy számolás implementálása és nyomonkövetése a tanulás során, animáció készítés a döntési határ alakulásáról a tanulás során, vagy a regularizásciós technikák implementálása és hatásuknak vizsgálata.